# My code is not cleaned
# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper`.

You will find more info about virtual environments [here](../2.python/1.python_fundamentals/02.Package-Managers/00.Package_Managers.md).

Activate it.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [52]:
# check the status_code using a condition and print appropriate messages (4 lines)
def status_to_text(status_code : int) -> str:
    status_dict = {
        # infos response
        100: "Continue",
        101: "Switching Protocols",
        102: "Processing",
        103: "Early Hints",
        # success response
        200: "OK",
        201: "Created",
        202: "Accepted",
        203: "Non-Authoritative Information",
        204: "No Content",
        205: "Reset Content",
        206: "Partial Content",
        207: "Multi-Status",
        208: "Already Reported",
        226: "IM Used",
        # redirection response
        300: "Multiple Choices",
        301: "Moved Permanently",
        302: "Found",
        303: "See Other",
        304: "Not Modified",
        305: "Use Proxy",
        306: "Switch Proxy",
        307: "Temporary Redirect",
        308: "Permanent Redirect",
        # client error response
        400: "Bad Request",
        401: "Unauthorized",
        402: "Payment Required",
        403: "Forbidden",
        404: "Not Found",
        405: "Method Not Allowed",
        406: "Not Acceptable",
        407: "Proxy Authentication Required",
        408: "Request Timeout",
        409: "Conflict",
        410: "Gone",
        411: "Length Required",
        412: "Precondition Failed",
        413: "Payload Too Large",
        414: "URI Too Long",
        415: "Unsupported Media Type",
        416: "Range Not Satisfiable",
        417: "Expectation Failed",
        418: "I'm a teapot",
        421: "Misdirected Request",
        422: "Unprocessable Entity",
        423: "Locked",
        424: "Failed Dependency",
        425: "Too Early",
        426: "Upgrade Required",
        428: "Precondition Required",
        429: "Too Many Requests",
        431: "Request Header Fields Too Large",
        451: "Unavailable For Legal Reasons",
        # server error response
        500: "Internal Server Error",
        501: "Not Implemented",
        502: "Bad Gateway",
        503: "Service Unavailable",
        504: "Gateway Timeout",
        505: "HTTP Version Not Supported",
        506: "Variant Also Negotiates",
        507: "Insufficient Storage",
        508: "Loop Detected",
        510: "Not Extended",
        511: "Network Authentication Required"
    }
    # find if the status code is in the keys
    if status_code in status_dict.keys():
        return str(status_code) + ": " + status_dict[status_code]
    else:
        return str(status_code) + ": Unknown or unofficial status code"


In [53]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = "/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
requ = requests.get(root_url + status_url)
   
print(root_url + status_url + " " + status_to_text(requ.status_code))
# print the content of the webpage if stautus is 200
if requ.status_code == 200:
    print(requ.content)

https://country-leaders.onrender.com/status 200: OK
b'"Alive"'


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [56]:
# Set the country_url variable (1 line)
country_url = "/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
requ =  requests.get(root_url + country_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = requ.json()

# display the request's status code and the countries variable (1 line)
print(root_url + country_url + " " + status_to_text(requ.status_code))

https://country-leaders.onrender.com/countries 403: Forbidden


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [57]:
# Set the session
session = requests.Session()

# Set the cookie_url variable (1 line)
cookie_url = "/cookie"

# Query the enpoint, set the cookies variable and display it (3 lines)
requ = requests.get(root_url + cookie_url)
cookies = requ.cookies
print(cookies)

<RequestsCookieJar[<Cookie user_cookie=b36cbbd6-24a5-4aad-97ea-fe374be28bb0 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [58]:
# query the /countries endpoint, assign the output to the countries variable (1 line) 
requ = requests.get(root_url + country_url, cookies=cookies)

# and check the status code and the countries variable's content (3 lines)
print(root_url + country_url + " " + status_to_text(requ.status_code))  
print(requ.json())

https://country-leaders.onrender.com/countries 200: OK
['us', 'ma', 'be', 'fr', 'ru']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [59]:
# query the /leaders endpoint, assign the output to the leaders variable (1 line)using the cookie
requ = requests.get(root_url + "/leaders", cookies=cookies)

# and check its content (4 lines)
print(root_url + "/leaders" + " " + status_to_text(requ.status_code))
print(requ.content)

https://country-leaders.onrender.com/leaders 404: Not Found
b'{"message":"Please specify a country"}'


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [62]:
# Set the leaders_url variable (1 line)
leaders_url = "/leaders"

# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (3 lines)
requ = requests.get(root_url + cookie_url)

cookies = requ.cookies
requ1 = requests.get(root_url + country_url, cookies=cookies)

countries = requ1.json()
requ2 = requests.get(root_url + leaders_url, cookies=cookies, params={"country": countries[0]})

# check the result (3 lines) 
print(root_url + leaders_url + " " + status_to_text(requ2.status_code))
print(requ2.json()[-1])

https://country-leaders.onrender.com/leaders 200: OK
{'id': 'Q36023', 'first_name': 'John', 'last_name': 'Coolidge', 'birth_date': '1872-07-04', 'death_date': '1933-01-05', 'place_of_birth': 'Plymouth Notch', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Calvin_Coolidge', 'start_mandate': '1923-08-02', 'end_mandate': '1929-03-04'}


### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each entry in a dictionary called `leaders_per_country` like this one: 
`{'fr':[{leader0}, {leader1},...],
'en':[{leader0}, {leader1},...],
...
}`.

In [64]:
# 5 lines
requ = requests.get(root_url + cookie_url)

cookies = requ.cookies
requ1 = requests.get(root_url + country_url, cookies=cookies)

countries = requ1.json()

lead = [requests.get(root_url + leaders_url, cookies=cookies, params={"country": country}).json() for country in countries]
leaders_dict = dict(map(lambda i,j : (i,j) , countries , lead)) 

print(leaders_dict["us"][5])

{'id': 'Q6279', 'first_name': 'Joe', 'last_name': 'Biden', 'birth_date': '1942-11-20', 'death_date': None, 'place_of_birth': "St. Mary's Hospital", 'wikipedia_url': 'https://en.wikipedia.org/wiki/Joe_Biden', 'start_mandate': '2021-01-20', 'end_mandate': None}


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [66]:
# 15 lines
def get_leaders():

    root_url = "https://country-leaders.onrender.com"
    cookie_url = "/cookie"
    country_url = "/countries"
    leaders_url = "/leaders"

    requ = requests.get(root_url + cookie_url)
    cookies = requ.cookies

    requ1 = requests.get(root_url + country_url, cookies=cookies)
    countries = requ1.json()

    lead = [requests.get(root_url + leaders_url, cookies=cookies, params={"country": country}).json() for country in countries]
    leaders_dict = dict(map(lambda i,j : (i,j) , countries , lead)) 
    
    return leaders_dict

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [67]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country["us"][5])

{'id': 'Q6279', 'first_name': 'Joe', 'last_name': 'Biden', 'birth_date': '1942-11-20', 'death_date': None, 'place_of_birth': "St. Mary's Hospital", 'wikipedia_url': 'https://en.wikipedia.org/wiki/Joe_Biden', 'start_mandate': '2021-01-20', 'end_mandate': None}


## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [68]:
# 3 lines
import requests
macron_url = 'https://fr.wikipedia.org/wiki/Emmanuel_Macron'
putin_url = "https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87"
req = requests.get("https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87")

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function to take a look. You will start the actual parsing in the next step.

In [69]:
# 3 lines
import bs4
soup = bs4.BeautifulSoup(req.content, "html.parser")
print(soup.find("title").text)

Путин, Владимир Владимирович — Википедия


In [70]:
def get_first_paragraph(url):

    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")

    for paragraph in soup.find_all('p'):

        if paragraph.find_all('b'):
            
            return paragraph.text

In [71]:
get_first_paragraph(url=macron_url)

"Emmanuel Macron (/ɛmanɥɛl makʁɔ̃/[e] Écouter), né le 21 décembre 1977 à Amiens est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis 2017.\n"

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [72]:
# 2 lines

def get_all_paragraphs(url):
    
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    return [paragraph.text for paragraph in soup.find_all('p')]

paragraphs = get_all_paragraphs(url=macron_url)


If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [73]:
# 6 lines
# there 2 things always present in the first paragraph 
# 1. all the words of the title but not concistant since for exemple russian words can change accent for  some reason
# 2. a text (the full title in bold)
# so we can get the first paragraph by fiding the first paragraph of the 
# page that contains all the words of the title in bold

import string
def get_first_paragraph(url):
    
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    
    title = soup.find('h1').text
    title = title.translate(str.maketrans('', '', string.punctuation))
    
    title_words = title.split()
    for paragraph in soup.find_all('p'):
        for bold in paragraph.find_all('b'):
            return paragraph.text
        
print(get_first_paragraph(url=macron_url))
print(get_first_paragraph(url=putin_url))


Emmanuel Macron (/ɛmanɥɛl makʁɔ̃/[e] Écouter), né le 21 décembre 1977 à Amiens est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis 2017.

Влади́мир Влади́мирович Пу́тин (род. 7 октября 1952, Ленинград, СССР) — российский государственный и политический деятель. Действующий президент Российской Федерации, председатель Государственного Совета Российской Федерации и верховный главнокомандующий Вооружёнными силами Российской Федерации с 7 мая 2012 года. Ранее занимал должность президента с 7 мая 2000 года по 7 мая 2008 года, также в 1999—2000 и 2008—2012 годах занимал должность председателя правительства Российской Федерации. Кандидат экономических наук (1997).



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [76]:
# 10 lines
import string
def get_first_paragraph(url):
    
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    #rtitle = soup.find('h1').text
    #remove punctuation character
    #rtitle = title.translate(str.maketrans('', '', string.punctuation))
    #rtitle_words = title.split()
    for paragraph in soup.find_all('p'):
        for bold in paragraph.find_all('b'):
            #if all(word in bold.text for word in title_words):
            return paragraph.text

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [77]:
# 4 lines
import re
# regular expression string to find things like this (/ɛmanɥɛl makʁɔ̃/[e] Écouter)
regex = "\(/.+/\[e\].*\)"
# compile the pattern
pattern = re.compile(regex)
# search
result = re.search(pattern, get_first_paragraph(url=macron_url))
print(result)

<re.Match object; span=(16, 45), match='(/ɛmanɥɛl makʁɔ̃/[e] Écouter)'>


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [78]:
# 10 lines
def get_first_paragraph(url):
    req = requests.get(url=url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    regex = "\ \(/.+/\[e\].*\)"
    pattern = re.compile(regex)
    for paragraph in soup.find_all('p'):
        for bold in paragraph.find_all('b'):
            return re.sub(pattern, "", paragraph.text)
    
print(get_first_paragraph(macron_url))

Emmanuel Macron, né le 21 décembre 1977 à Amiens est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis 2017.



Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [102]:
import re


def get_first_paragraph(url):
    
    regex_list = ["\ *\(/.+/\[e\].*\)" , 
                  "\ *\(/.+/.*\)",
                  "\[[0-9]+\]" , 
                  "[\n]", 
                  "[\t]", 
                  "[\xa0]"]
    req = requests.get(url=url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")

    for paragraph in soup.find_all('p'):

        for bold in paragraph.find_all('b'):

            text = paragraph.text

            for regex in regex_list:

                pattern = re.compile(regex)
                text = re.sub(pattern, "", text)

            return text

def get_all_paragraphs(url):

    req = requests.get(url=url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    regex_list = ["\ *\(/.+/\[e\].*\)" , 
                  "\ *\(/.+/.*\)",
                  "\[[0-9]+\]" , 
                  "[\n]", 
                  "[\t]", 
                  "[\xa0]"]
    paragraphs = []

    for paragraph in soup.find_all('p'):

        text = paragraph.text

        for regex in regex_list:

            pattern = re.compile(regex)
            text = re.sub(pattern, "", text)
            
        paragraphs.append(text)

    return paragraphs

print(get_first_paragraph(macron_url))
print(get_first_paragraph(putin_url))

Emmanuel Macron, né le 21 décembre 1977 à Amiens est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis 2017.
Влади́мир Влади́мирович Пу́тин (род. 7 октября 1952, Ленинград, СССР)— российский государственный и политический деятель. Действующий президент Российской Федерации, председатель Государственного Совета Российской Федерации и верховный главнокомандующий Вооружёнными силами Российской Федерации с 7мая 2012 года. Ранее занимал должность президента с 7мая 2000года по 7мая 2008 года, также в 1999—2000 и 2008—2012 годах занимал должность председателя правительства Российской Федерации. Кандидат экономических наук (1997).


### Caching
Good job! This function will be called many times. First because there are many leaders to loop through. Second because you are developing your code and are likely to re-run your loop multiple times. Third because the API might change and contain new leaders and you might want to update your data periodically.

For all these reasons, it can be benificial to *cache* the result of the `get_first_paragraph()` function so that if it called with the same parameter, you expect the same output and thus do not need to query Wikipedia another time.

You will use the [lru_cache](https://docs.python.org/3/library/functools.html#functools.lru_cache) decorator from the `functools` module. It is a tiny code change that will speed things up big time. You can use a `maxsize` of None. If you have questions about decorators, you will find more information [here](../../2.python/2.python_advanced/07.Decorator/decorators.ipynb)

In [100]:
import functools
import re

#use lru cache to speedup the code
@functools.lru_cache(maxsize=None)
def get_first_paragraph(url):
    
    regex_list = ["\ *\(/.+/\[e\].*\)" , 
                  "\ *\(/.+/.*\)",
                  "\[[0-9]+\]" , 
                  "[\n]", 
                  "[\t]", 
                  "[\xa0]"]
    req = requests.get(url=url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")

    for paragraph in soup.find_all('p'):

        for bold in paragraph.find_all('b'):

            text = paragraph.text

            for regex in regex_list:

                pattern = re.compile(regex)
                text = re.sub(pattern, "", text)

            return text
    

Let's check the speedup!

In [101]:
import time
# Just run the cell. 

start = time.time()
get_first_paragraph(macron_url)
end = time.time()
print("Time to get the first paragraph of wikipedia page: ", end - start)

start = time.time()
get_first_paragraph(macron_url)
end = time.time()
print("Time to get the first paragraph of wikipedia page: ", end - start)

Time to get the first paragraph of wikipedia page:  1.1304445266723633
Time to get the first paragraph of wikipedia page:  4.291534423828125e-05


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [98]:
# Just ctrl c v to avoid rerun the entire notebook
import functools
import re
import time
import bs4
import requests

def get_leaders():

    root_url = "https://country-leaders.onrender.com"
    cookie_url = "/cookie"
    country_url = "/countries"
    leaders_url = "/leaders"

    requ = requests.get(root_url + cookie_url)
    cookies = requ.cookies

    requ1 = requests.get(root_url + country_url, cookies=cookies)
    countries = requ1.json()

    lead = [requests.get(root_url + leaders_url, cookies=cookies, params={"country": country}).json() for country in countries]
    
    leaders_per_country = dict(map(lambda i,j : (i,j) , countries , lead)) 

    for country, leaders in leaders_per_country.items():

        for leader in leaders:

            try:
                leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = get_first_paragraph(leader["wikipedia_url"])
                
            except Exception as e:
                leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = "Error"
                print(str(e))
                continue

    return leaders_per_country

def get_first_paragraph(url):
    
    regex_list = ["\ *\(/.+/\[e\].*\)" , 
                  "\ *\(/.+/.*\)",
                  "\[[0-9]+\]" , 
                  "[\n]", 
                  "[\t]", 
                  "[\xa0]"]
    req = requests.get(url=url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")

    for paragraph in soup.find_all('p'):

        for bold in paragraph.find_all('b'):

            text = paragraph.text

            for regex in regex_list:

                pattern = re.compile(regex)
                text = re.sub(pattern, "", text)

            return text
        
        

In [99]:
# Check the output of your function (2 lines)
leaders_per_country = get_leaders()
print(leaders_per_country["us"][5])

{'id': 'Q6279', 'first_name': 'Joe', 'last_name': 'Biden', 'birth_date': '1942-11-20', 'death_date': None, 'place_of_birth': "St. Mary's Hospital", 'wikipedia_url': 'https://en.wikipedia.org/wiki/Joe_Biden', 'start_mandate': '2021-01-20', 'end_mandate': None, 'first_paragraph': 'Joseph Robinette Biden Jr. is an American politician who is the 46th and current president of the United States. A member of the Democratic Party, he previously served as the 47th vice president from 2009 to 2017 under President Barack Obama, and represented Delaware in the United States Senate from 1973 to 2009.'}


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function to check if the `status_code` is a cookie error. In which case, get new ones and query the api again.

If not, you may have a fast connection or a bit of luck. Do it anyway.

In [95]:
import functools
import re
import time
import bs4
import requests
import warnings
from urllib3.exceptions import NewConnectionError


def status_to_text(status_code : int) -> str:
    status_dict = {
        # infos response
        100: "Continue",
        101: "Switching Protocols",
        102: "Processing",
        103: "Early Hints",
        # success response
        200: "OK",
        201: "Created",
        202: "Accepted",
        203: "Non-Authoritative Information",
        204: "No Content",
        205: "Reset Content",
        206: "Partial Content",
        207: "Multi-Status",
        208: "Already Reported",
        226: "IM Used",
        # redirection response
        300: "Multiple Choices",
        301: "Moved Permanently",
        302: "Found",
        303: "See Other",
        304: "Not Modified",
        305: "Use Proxy",
        306: "Switch Proxy",
        307: "Temporary Redirect",
        308: "Permanent Redirect",
        # client error response
        400: "Bad Request",
        401: "Unauthorized",
        402: "Payment Required",
        403: "Forbidden",
        404: "Not Found",
        405: "Method Not Allowed",
        406: "Not Acceptable",
        407: "Proxy Authentication Required",
        408: "Request Timeout",
        409: "Conflict",
        410: "Gone",
        411: "Length Required",
        412: "Precondition Failed",
        413: "Payload Too Large",
        414: "URI Too Long",
        415: "Unsupported Media Type",
        416: "Range Not Satisfiable",
        417: "Expectation Failed",
        418: "I'm a teapot",
        421: "Misdirected Request",
        422: "Unprocessable Entity",
        423: "Locked",
        424: "Failed Dependency",
        425: "Too Early",
        426: "Upgrade Required",
        428: "Precondition Required",
        429: "Too Many Requests",
        431: "Request Header Fields Too Large",
        451: "Unavailable For Legal Reasons",
        # server error response
        500: "Internal Server Error",
        501: "Not Implemented",
        502: "Bad Gateway",
        503: "Service Unavailable",
        504: "Gateway Timeout",
        505: "HTTP Version Not Supported",
        506: "Variant Also Negotiates",
        507: "Insufficient Storage",
        508: "Loop Detected",
        510: "Not Extended",
        511: "Network Authentication Required"
    }
    # find if the status code is in the keys
    if status_code in status_dict.keys():
        return str(status_code) + ": " + status_dict[status_code]
    else:
        return str(status_code) + ": Unknown or unofficial status code"


def get_leaders():

    root_url = "https://country-leaders.onrender.com"
    cookie_url = "/cookie"
    country_url = "/countries"
    leaders_url = "/leaders"

    requ = requests.get(root_url + cookie_url)
    cookies = requ.cookies

    requ1 = requests.get(root_url + country_url, cookies=cookies)
    countries = requ1.json()

    lead = [requests.get(root_url + leaders_url, cookies=cookies, params={"country": country}).json() for country in countries]
    
    leaders_per_country = dict(map(lambda i,j : (i,j) , countries , lead)) 

    for country, leaders in leaders_per_country.items():

        for leader in leaders:

            try:
                leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = get_first_paragraph(leader["wikipedia_url"])

            except NewConnectionError as nce:
                leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = "NewConnectionError" + str(nce)
                print(str(nce))
                continue

            except Exception as e:
                leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = "Error" + str(nce)
                print(str(e))
                continue

    return leaders_per_country


def get_first_paragraph(url):
    
    regex_list = ["\ *\(/.+/\[e\].*\)" , 
                  "\ *\(/.+/.*\)",
                  "\[[0-9]+\]" , 
                  "[\n]", 
                  "[\t]", 
                  "[\xa0]"]

    req = requests.get(url=url)
    
    # print the content of the webpage if stautus is 200
    if req.status_code != 200:
        
        message = root_url + status_url + " " + status_to_text(req.status_code)
        warnings.warn(message)
        return message

    soup = bs4.BeautifulSoup(req.text, "html.parser")

    for paragraph in soup.find_all('p'):

        for bold in paragraph.find_all('b'):

            text = paragraph.text

            for regex in regex_list:

                pattern = re.compile(regex)
                text = re.sub(pattern, "", text)

            return text


Check the output of your function again.

In [96]:
import time
# Check the output of your function (1 line)

start = time.time()
leaders_per_country = get_leaders()
end = time.time()
print("Time to scrap: ", end - start)

Time to scrap:  83.40544772148132


In [97]:
print(leaders_per_country["us"][5])

{'id': 'Q6279', 'first_name': 'Joe', 'last_name': 'Biden', 'birth_date': '1942-11-20', 'death_date': None, 'place_of_birth': "St. Mary's Hospital", 'wikipedia_url': 'https://en.wikipedia.org/wiki/Joe_Biden', 'start_mandate': '2021-01-20', 'end_mandate': None, 'first_paragraph': 'Joseph Robinette Biden Jr. is an American politician who is the 46th and current president of the United States. A member of the Democratic Party, he previously served as the 47th vice president from 2009 to 2017 under President Barack Obama, and represented Delaware in the United States Senate from 1973 to 2009.'}


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.
Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [116]:
import functools
import re
import time
import bs4
import requests
import warnings
from requests import Session
from urllib3.exceptions import NewConnectionError

@functools.lru_cache(maxsize=None)
def status_to_text(status_code : int) -> str:
    status_dict = {
        # infos response
        100: "Continue",
        101: "Switching Protocols",
        102: "Processing",
        103: "Early Hints",
        # success response
        200: "OK",
        201: "Created",
        202: "Accepted",
        203: "Non-Authoritative Information",
        204: "No Content",
        205: "Reset Content",
        206: "Partial Content",
        207: "Multi-Status",
        208: "Already Reported",
        226: "IM Used",
        # redirection response
        300: "Multiple Choices",
        301: "Moved Permanently",
        302: "Found",
        303: "See Other",
        304: "Not Modified",
        305: "Use Proxy",
        306: "Switch Proxy",
        307: "Temporary Redirect",
        308: "Permanent Redirect",
        # client error response
        400: "Bad Request",
        401: "Unauthorized",
        402: "Payment Required",
        403: "Forbidden",
        404: "Not Found",
        405: "Method Not Allowed",
        406: "Not Acceptable",
        407: "Proxy Authentication Required",
        408: "Request Timeout",
        409: "Conflict",
        410: "Gone",
        411: "Length Required",
        412: "Precondition Failed",
        413: "Payload Too Large",
        414: "URI Too Long",
        415: "Unsupported Media Type",
        416: "Range Not Satisfiable",
        417: "Expectation Failed",
        418: "I'm a teapot",
        421: "Misdirected Request",
        422: "Unprocessable Entity",
        423: "Locked",
        424: "Failed Dependency",
        425: "Too Early",
        426: "Upgrade Required",
        428: "Precondition Required",
        429: "Too Many Requests",
        431: "Request Header Fields Too Large",
        451: "Unavailable For Legal Reasons",
        # server error response
        500: "Internal Server Error",
        501: "Not Implemented",
        502: "Bad Gateway",
        503: "Service Unavailable",
        504: "Gateway Timeout",
        505: "HTTP Version Not Supported",
        506: "Variant Also Negotiates",
        507: "Insufficient Storage",
        508: "Loop Detected",
        510: "Not Extended",
        511: "Network Authentication Required"
    }
    # find if the status code is in the keys
    if status_code in status_dict.keys():
        return str(status_code) + ": " + status_dict[status_code]
    else:
        return str(status_code) + ": Unknown or unofficial status code"


@functools.lru_cache(maxsize=None)
def get_lead(sess):

    root_url = "https://country-leaders.onrender.com"
    cookie_url = "/cookie"
    country_url = "/countries"
    leaders_url = "/leaders"

    requ = sess.get(root_url + cookie_url)
    cookies = requ.cookies

    requ1 = sess.get(root_url + country_url, cookies=cookies)
    countries = requ1.json()

    lead = [sess.get(root_url + leaders_url, 
                        cookies=cookies, 
                        params={"country": country}).json() for country in countries]

    leaders_dict = dict(map(lambda i,j : (i,j) , countries , lead)) 

    return leaders_dict


@functools.lru_cache(maxsize=None)
def get_first_paragraph(url,sess):
    
    regex_list = ["\ *\(/.+/\[e\].*\)" , 
                  "\ *\(/.+/.*\)",
                  "\[[0-9]+\]" , 
                  "[\n]", 
                  "[\t]", 
                  "[\xa0]"]

    req = sess.get(url=url)
    
    # print the content of the webpage if stautus is 200
    if req.status_code != 200:
        message = root_url + status_url + " " + status_to_text(req.status_code)
        warnings.warn(message)
        return message

    soup = bs4.BeautifulSoup(req.text, "html.parser")

    for paragraph in soup.find_all('p'):

        for bold in paragraph.find_all('b'):

            text = paragraph.text

            for regex in regex_list:

                pattern = re.compile(regex)
                text = re.sub(pattern, "", text)

            return text
            

def get_leaders():
    with Session() as session:
        
        leaders_per_country = get_lead(session)

        for country, leaders in leaders_per_country.items():

            for leader in leaders:

                try:
                    leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = get_first_paragraph(leader["wikipedia_url"],session)

                except NewConnectionError as nce:
                    leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = "NewConnectionError" + str(nce)
                    print(str(nce))
                    continue

                except Exception as e:
                    leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = "Error" + str(nce)
                    print(str(e))
                    continue
            
    return leaders_per_country


Test your new functions.



In [114]:
# Just run it.
import time

start = time.time()
leaders_per_country = get_leaders()
end = time.time()
print("Time to scrap: ", end - start)

Time to scrap:  55.0427930355072


Using a Session makes the first call to `get_leaders()` faster however, for technical reasons, the `@lru_cache` decorator no longer works. 

Problem: The `session` parameter is not *hashable* and since the cache is a dictionary using all the parameters as key, this poses a problem.

Solution: Create our own cache that only cares about the url.

The code for this is provided. You should only read it.

In [31]:
cache = {}
def hashable_cache(f):
    def inner(url, session):
        if url not in cache:
            cache[url] = f(url, session)
        return cache[url]
    return inner

Overwrite the `get_first_paragraph()` function to use the new decorator like this: `@hashable_cache`.


In [117]:
import functools
import re
import time
import bs4
import requests
import warnings
from urllib3.exceptions import NewConnectionError

@functools.lru_cache(maxsize=None)
def status_to_text(status_code : int) -> str:
    status_dict = {
        # infos response
        100: "Continue",
        101: "Switching Protocols",
        102: "Processing",
        103: "Early Hints",
        # success response
        200: "OK",
        201: "Created",
        202: "Accepted",
        203: "Non-Authoritative Information",
        204: "No Content",
        205: "Reset Content",
        206: "Partial Content",
        207: "Multi-Status",
        208: "Already Reported",
        226: "IM Used",
        # redirection response
        300: "Multiple Choices",
        301: "Moved Permanently",
        302: "Found",
        303: "See Other",
        304: "Not Modified",
        305: "Use Proxy",
        306: "Switch Proxy",
        307: "Temporary Redirect",
        308: "Permanent Redirect",
        # client error response
        400: "Bad Request",
        401: "Unauthorized",
        402: "Payment Required",
        403: "Forbidden",
        404: "Not Found",
        405: "Method Not Allowed",
        406: "Not Acceptable",
        407: "Proxy Authentication Required",
        408: "Request Timeout",
        409: "Conflict",
        410: "Gone",
        411: "Length Required",
        412: "Precondition Failed",
        413: "Payload Too Large",
        414: "URI Too Long",
        415: "Unsupported Media Type",
        416: "Range Not Satisfiable",
        417: "Expectation Failed",
        418: "I'm a teapot",
        421: "Misdirected Request",
        422: "Unprocessable Entity",
        423: "Locked",
        424: "Failed Dependency",
        425: "Too Early",
        426: "Upgrade Required",
        428: "Precondition Required",
        429: "Too Many Requests",
        431: "Request Header Fields Too Large",
        451: "Unavailable For Legal Reasons",
        # server error response
        500: "Internal Server Error",
        501: "Not Implemented",
        502: "Bad Gateway",
        503: "Service Unavailable",
        504: "Gateway Timeout",
        505: "HTTP Version Not Supported",
        506: "Variant Also Negotiates",
        507: "Insufficient Storage",
        508: "Loop Detected",
        510: "Not Extended",
        511: "Network Authentication Required"
    }
    # find if the status code is in the keys
    if status_code in status_dict.keys():
        return str(status_code) + ": " + status_dict[status_code]
    else:
        return str(status_code) + ": Unknown or unofficial status code"


@functools.lru_cache(maxsize=None)
def get_lead(sess):

    root_url = "https://country-leaders.onrender.com"
    cookie_url = "/cookie"
    country_url = "/countries"
    leaders_url = "/leaders"

    requ = sess.get(root_url + cookie_url)
    cookies = requ.cookies

    requ1 = sess.get(root_url + country_url, cookies=cookies)
    countries = requ1.json()

    lead = [sess.get(root_url + leaders_url, 
                        cookies=cookies, 
                        params={"country": country}).json() for country in countries]

    leaders_dict = dict(map(lambda i,j : (i,j) , countries , lead)) 

    return leaders_dict


@hashable_cache
def get_first_paragraph(url,sess):
    
    regex_list = ["\ *\(/.+/\[e\].*\)" , 
                  "\ *\(/.+/.*\)",
                  "\[[0-9]+\]" , 
                  "[\n]", 
                  "[\t]", 
                  "[\xa0]"]

    req = sess.get(url=url)
    
    # print the content of the webpage if stautus is 200
    if req.status_code != 200:
        message = root_url + status_url + " " + status_to_text(req.status_code)
        warnings.warn(message)
        return message

    soup = bs4.BeautifulSoup(req.text, "html.parser")

    for paragraph in soup.find_all('p'):

        for bold in paragraph.find_all('b'):

            text = paragraph.text

            for regex in regex_list:

                pattern = re.compile(regex)
                text = re.sub(pattern, "", text)

            return text
            

def get_leaders():
    with Session() as session:
        
        leaders_per_country = get_lead(session)

        for country, leaders in leaders_per_country.items():

            for leader in leaders:

                try:
                    leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = get_first_paragraph(leader["wikipedia_url"],session)

                except NewConnectionError as nce:
                    leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = "NewConnectionError" + str(nce)
                    print(str(nce))
                    continue

                except Exception as e:
                    leaders_per_country[country][leaders.index(leader)]["first_paragraph"] = "Error" + str(nce)
                    print(str(e))
                    continue
            
    return leaders_per_country


Check that the cache is working again.

In [125]:
# Just run it.
import time

start = time.time()
leaders_per_country = get_leaders()
end = time.time()
print("Time to scrap: ", end - start)

Time to scrap:  1.059046983718872


## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Make a tour in our [file handling section](../../2.python/2.python_advanced/04.File-handling/file_handling.ipynb) to know more about how to read and write files using Python.

In [126]:
import json

with open("leaders.json", "w") as the_file:
    json.dump(leaders_per_country, the_file)


Make sure the file can be read back. Write the code to read the file.

In [127]:
import json
with open("leaders.json", "r") as the_file:
    leaders_per_country = json.load(the_file)
print(leaders_per_country[list(leaders_per_country.keys())[0]][0]["first_paragraph"])

George Washington (February 22, 1732[b]– December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led the Patriot forces to victory in the American Revolutionary War and served as the president of the Constitutional Convention of 1787, which created the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the formative days of the country.


Make a function `save(leaders_per_country)` to call this code easily.

In [128]:
import json
def save(leaders_per_country):
    with open("leaders.json", "w") as the_file:
        json.dump(leaders_per_country, the_file)

In [129]:
# Call the function (1 line)
save(leaders_per_country)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!